In [1]:
# importing required libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import pickle
from os import path

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_train = pd.read_csv("/content/drive/MyDrive/dataset_unsw/UNSW_NB15_testing-set.csv")
df_test = pd.read_csv("/content/drive/MyDrive/dataset_unsw/UNSW_NB15_training-set.csv")

In [4]:
#Dropping the last columns of testing set label and id
df_train = df_train.drop('id', 1)
df_train = df_train.drop('label', 1)
df_train.shape

<ipython-input-4-cd5ec6db61cd>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_train = df_train.drop('id', 1)
<ipython-input-4-cd5ec6db61cd>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_train = df_train.drop('label', 1)


(175341, 43)

In [5]:
#Dropping the last columns of testing set label and id
df_test = df_test.drop('id', 1)
df_test = df_test.drop('label', 1)
df_test.shape

<ipython-input-5-33eea86c10e9>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_test = df_test.drop('id', 1)
<ipython-input-5-33eea86c10e9>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_test = df_test.drop('label', 1)


(82332, 43)

In [6]:
# train dataset select numeric colom

num_col_train = df_train.select_dtypes(include='number').columns

# selecting categorical data attributes
cat_col_train = df_train.columns.difference(num_col_train)
cat_col_train = cat_col_train[1:]
cat_col_train

Index(['proto', 'service', 'state'], dtype='object')

In [7]:
# train dataset select numeric colom

num_col_test = df_test.select_dtypes(include='number').columns

# selecting categorical data attributes
cat_col_test = df_test.columns.difference(num_col_test)
cat_col_test = cat_col_test[1:]
cat_col_test

Index(['proto', 'service', 'state'], dtype='object')

In [8]:
#One-hot encoding
def one_hot(df, cols):

    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(each, 1)
    return df

In [9]:
#Applying one hot encoding to train data
df_train = one_hot(df_train,cat_col_train)

<ipython-input-8-40610ed0d857>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
<ipython-input-8-40610ed0d857>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
<ipython-input-8-40610ed0d857>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)


In [10]:
#Applying one hot encoding to test data
df_test = one_hot(df_test,cat_col_test)

<ipython-input-8-40610ed0d857>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
<ipython-input-8-40610ed0d857>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
<ipython-input-8-40610ed0d857>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)


In [11]:
# Find the columns that are in df_train but not in df_test
extra_columns = set(df_train.columns) - set(df_test.columns)

In [12]:
# Convert the set to a list if needed
extra_columns_list = list(extra_columns)

print("Columns in df_train but not in df_test:")
print(extra_columns_list)

Columns in df_train but not in df_test:
['proto_icmp', 'state_URN', 'state_PAR', 'state_no', 'state_ECO', 'proto_rtp']


In [13]:
# Get the set of column names in df_train
column_names_train = set(df_train.columns)

# Get the set of column names in df_test
column_names_test = set(df_test.columns)

# Find the common column names
common_column_names = column_names_train.intersection(column_names_test)

# Get the count of common column names
count_common_column_names = len(common_column_names)

print(f"Number of columns with the same name in both dataframes: {count_common_column_names}")

Number of columns with the same name in both dataframes: 189


In [14]:
# Get the common column names
common_column_names = df_train.columns.intersection(df_test.columns)

# Keep only the common columns in both dataframes
df_train = df_train[common_column_names]
df_test = df_test[common_column_names]

In [15]:
# take attack_colom into another variable temp
temp_1 = df_train.pop('attack_cat')
temp_2 = df_test.pop('attack_cat')

In [16]:
#Function to min-max normalize
def normalize(df, cols):

    result = df.copy() # do not touch the original df
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value > min_value:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [17]:
#Normalizing training set
new_train_df = normalize(df_train,df_train.columns)
new_train_df

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,state_RST
0,2.024634e-03,0.000520,0.000364,0.000018,0.000012,0.000074,0.988235,1.000000,2.364553e-06,0.000379,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.083170e-02,0.001352,0.003463,0.000054,0.002867,0.000078,0.243137,0.992126,1.401989e-06,0.022458,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2.705215e-02,0.000728,0.001458,0.000026,0.000900,0.000014,0.243137,0.992126,2.625704e-07,0.002717,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2.802737e-02,0.001144,0.001093,0.000046,0.000053,0.000014,0.243137,0.992126,4.576117e-07,0.000150,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,7.490901e-03,0.000936,0.000547,0.000039,0.000018,0.000033,0.996078,0.992126,1.429776e-06,0.000178,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,1.500000e-07,0.000104,0.000000,0.000007,0.000000,0.111111,0.996078,0.000000,8.461366e-03,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
175337,8.429368e-03,0.000936,0.000729,0.000046,0.000024,0.000034,0.996078,0.992126,1.473996e-06,0.000219,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
175338,1.500000e-07,0.000104,0.000000,0.000007,0.000000,0.111111,0.996078,0.000000,8.461366e-03,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
175339,1.500000e-07,0.000104,0.000000,0.000007,0.000000,0.111111,0.996078,0.000000,8.461366e-03,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [18]:
#Normalizing testing set
new_test_df = normalize(df_test,df_test.columns)
new_test_df

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,state_RST
0,1.833334e-07,0.000094,0.000000,0.000033,0.000000,0.090909,0.996078,0.000000,0.034238,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.333334e-07,0.000094,0.000000,0.000121,0.000000,0.125000,0.996078,0.000000,0.167236,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,8.333335e-08,0.000094,0.000000,0.000073,0.000000,0.200000,0.996078,0.000000,0.162187,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.000000e-07,0.000094,0.000000,0.000061,0.000000,0.166667,0.996078,0.000000,0.113895,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.666667e-07,0.000094,0.000000,0.000146,0.000000,0.100000,0.996078,0.000000,0.161427,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,8.333335e-08,0.000094,0.000000,0.000006,0.000000,0.200000,0.996078,0.000000,0.015793,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
82328,1.843502e-02,0.001785,0.000726,0.001257,0.000024,0.000024,0.996078,0.996047,0.000024,0.000108,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
82329,0.000000e+00,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
82330,0.000000e+00,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
#Appending class column to training set
new_train_df["Class"] = temp_1
new_train_df.head()

<ipython-input-19-8ec02d33ca04>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_train_df["Class"] = temp_1


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_smtp,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,state_RST,Class
0,0.002025,0.000520,0.000364,0.000018,0.000012,0.000074,0.988235,1.000000,2.364553e-06,0.000379,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Normal
1,0.010832,0.001352,0.003463,0.000054,0.002867,0.000078,0.243137,0.992126,1.401989e-06,0.022458,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Normal
2,0.027052,0.000728,0.001458,0.000026,0.000900,0.000014,0.243137,0.992126,2.625704e-07,0.002717,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Normal
3,0.028027,0.001144,0.001093,0.000046,0.000053,0.000014,0.243137,0.992126,4.576117e-07,0.000150,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Normal
4,0.007491,0.000936,0.000547,0.000039,0.000018,0.000033,0.996078,0.992126,1.429776e-06,0.000178,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Normal


In [20]:
#Appending class column to training set
new_test_df["Class"] = temp_2
new_test_df.head()

<ipython-input-20-c7b3a1d36faf>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_test_df["Class"] = temp_2


,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_smtp,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,state_RST,Class
0,1.833334e-07,0.000094,0.0,0.000033,0.0,0.090909,0.996078,0.0,0.034238,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Normal
1,1.333334e-07,0.000094,0.0,0.000121,0.0,0.125000,0.996078,0.0,0.167236,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Normal
2,8.333335e-08,0.000094,0.0,0.000073,0.0,0.200000,0.996078,0.0,0.162187,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Normal
3,1.000000e-07,0.000094,0.0,0.000061,0.0,0.166667,0.996078,0.0,0.113895,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Normal
4,1.666667e-07,0.000094,0.0,0.000146,0.0,0.100000,0.996078,0.0,0.161427,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,Normal


In [21]:
# Create a mapping dictionary
mapping = {
    'Normal': 0,
    'Backdoor': 1,
    'Analysis': 2,
    'Fuzzers': 3,
    'Shellcode': 4,
    'Reconnaissance': 5,
    'Exploits': 6,
    'DoS': 7,
    'Worms': 8,
    'Generic': 9
}

In [22]:
# Replace the values in the existing column with numerical labels
new_train_df['Class'] = new_train_df['Class'].map(mapping)

In [23]:
# Replace the values in the existing column with numerical labels
new_test_df['Class'] = new_test_df['Class'].map(mapping)
new_test_df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_smtp,service_snmp,service_ssh,service_ssl,state_CON,state_FIN,state_INT,state_REQ,state_RST,Class
0,1.833334e-07,0.000094,0.0,0.000033,0.0,0.090909,0.996078,0.0,0.034238,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,1.333334e-07,0.000094,0.0,0.000121,0.0,0.125000,0.996078,0.0,0.167236,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,8.333335e-08,0.000094,0.0,0.000073,0.0,0.200000,0.996078,0.0,0.162187,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,1.000000e-07,0.000094,0.0,0.000061,0.0,0.166667,0.996078,0.0,0.113895,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,1.666667e-07,0.000094,0.0,0.000146,0.0,0.100000,0.996078,0.0,0.161427,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [24]:
X = new_train_df.drop(columns=['Class'])
y = new_train_df['Class']

In [25]:
X_test_dataset = new_test_df.drop(columns=['Class'])
y_test_dataset = new_test_df['Class']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.feature_selection import SelectKBest
def fcbf(X, y, k):
    """
    Fast Correlation-Based Feature Selection (FCBF) algorithm
    :param X: input features matrix
    :param y: target vector
    :param k: number of features to select
    :return: indices of the k selected features
    """
    n_features = X.shape[1]
    sigma = np.zeros(n_features)
    corr = np.zeros((n_features, n_features))
    for i in range(n_features):
        sigma[i] = np.std(X[:, i])
        for j in range(i+1, n_features):
            corr[i,j] = abs(np.corrcoef(X[:,i], X[:,j], rowvar=False)[0,1])
            corr[j,i] = corr[i,j]
    mi = np.zeros(n_features)
    for i in range(n_features):
        mi[i] = sum([corr[i,j]*np.log2(corr[i,j]/(sigma[i]*sigma[j])) for j in range(n_features)])
    k_best = SelectKBest(k='all').fit(X, y)
    F = k_best.scores_
    F_ = np.zeros(n_features)
    for i in range(n_features):
        F_[i] = (n_features-2)*mi[i]/sum([mi[j] for j in k_best.get_support(indices=True) if j!=i])
    idx = np.argsort(F_)[::-1][:k]
    return idx

In [30]:
k = 20
idx = fcbf(X_train.values, y_train.values, k)

<ipython-input-29-e491e55bde1d>:20: RuntimeWarning: divide by zero encountered in log2
  mi[i] = sum([corr[i,j]*np.log2(corr[i,j]/(sigma[i]*sigma[j])) for j in range(n_features)])
<ipython-input-29-e491e55bde1d>:20: RuntimeWarning: invalid value encountered in double_scalars
  mi[i] = sum([corr[i,j]*np.log2(corr[i,j]/(sigma[i]*sigma[j])) for j in range(n_features)])


In [31]:
X_train_fs = X_train.iloc[:, idx]
X_test_fs = X_test_dataset.iloc[:, idx]

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust hyperparameters like n_estimators
rf_model.fit(X_train_fs, y_train)

RandomForestClassifier(random_state=42)

In [33]:
y_pred_rf = rf_model.predict(X_test_fs)

In [34]:
accuracy_rf = accuracy_score(y_test_dataset, y_pred_rf)
print(f'Validation Accuracy: {accuracy_rf * 100:.2f}%')

Validation Accuracy: 45.19%
